In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Import all the libraries we will need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [81]:
# Read train data and test data
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Xente Fraud Detection Challenge/training.csv')
test_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Xente Fraud Detection Challenge/test.csv')
# view first few entries of train_df
train_df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [199]:
train_df.tail()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-13T09:54:09Z,2,0
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-02-13T09:54:25Z,2,0
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2019-02-13T09:54:35Z,2,0
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3,3000.0,3000,2019-02-13T10:01:10Z,2,0
95661,TransactionId_35670,BatchId_29317,AccountId_4841,SubscriptionId_3829,CustomerId_1709,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:01:28Z,2,0


In [83]:
# Print out what test_df looks like
test_df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [84]:
# Checking for missing values for train_df
train_df.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [85]:
# Checking for missing values for test_df
test_df.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
dtype: int64

In [86]:
# Checking size of both data files
print(train_df.shape)
print(test_df.shape)

(95662, 16)
(45019, 15)


In [0]:
# Combining both data frame to one dataframe
df = pd.concat([train_df, test_df], axis=0, sort=True)

In [88]:
# Check first few entry of df(combined train_df and test_df)
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644


In [89]:
# Checking size of df
df.shape

(140681, 16)

In [90]:
# Check Last 5 entry of df(combined train_df and test_df)
df.tail()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value
45014,AccountId_4841,-5000.0,BatchId_117348,ChannelId_3,256,UGX,CustomerId_5225,NaN,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_116198,2019-03-14T23:56:28Z,5000
45015,AccountId_4803,10000.0,BatchId_53630,ChannelId_3,256,UGX,CustomerId_5272,NaN,1,airtime,ProductId_10,ProviderId_3,SubscriptionId_1738,TransactionId_24829,2019-03-14T23:57:38Z,10000
45016,AccountId_4817,10000.0,BatchId_112209,ChannelId_3,256,UGX,CustomerId_5286,NaN,1,airtime,ProductId_10,ProviderId_3,SubscriptionId_4613,TransactionId_102920,2019-03-14T23:59:17Z,10000
45017,AccountId_4841,-5000.0,BatchId_58734,ChannelId_3,256,UGX,CustomerId_5286,NaN,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_128439,2019-03-14T23:59:39Z,5000
45018,AccountId_4593,3500.0,BatchId_98308,ChannelId_3,256,UGX,CustomerId_5057,NaN,4,airtime,ProductId_3,ProviderId_5,SubscriptionId_3866,TransactionId_45927,2019-03-14T23:59:51Z,3500


In [0]:
# create a year column from TransactionStartTime column
df['Year'] = pd.DatetimeIndex(df['TransactionStartTime']).year

In [92]:
# check the created new column Year
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value,Year
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000,2018
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20,2018
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500,2018
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800,2018
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644,2018


In [93]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value,Year
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000,2018
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20,2018
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500,2018
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800,2018
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644,2018


In [0]:
# Drop the TransactionStartTime  column as year alone is enough useful
df.drop(['TransactionStartTime'], axis = 1, inplace = True)

In [95]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,Value,Year
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,1000,2018
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,20,2018
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,500,2018
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,21800,2018
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,644,2018


In [96]:
# Check Currency Code
df['CurrencyCode'].describe()

count     140681
unique         1
top          UGX
freq      140681
Name: CurrencyCode, dtype: object

In [97]:
# Check Currency Code
df['CountryCode'].describe

<bound method NDFrame.describe of 0        256
1        256
2        256
3        256
4        256
5        256
6        256
7        256
8        256
9        256
10       256
11       256
12       256
13       256
14       256
15       256
16       256
17       256
18       256
19       256
20       256
21       256
22       256
23       256
24       256
25       256
26       256
27       256
28       256
29       256
        ... 
44989    256
44990    256
44991    256
44992    256
44993    256
44994    256
44995    256
44996    256
44997    256
44998    256
44999    256
45000    256
45001    256
45002    256
45003    256
45004    256
45005    256
45006    256
45007    256
45008    256
45009    256
45010    256
45011    256
45012    256
45013    256
45014    256
45015    256
45016    256
45017    256
45018    256
Name: CountryCode, Length: 140681, dtype: int64>

It looks like we have one CountryCode thrughout

In [0]:
# Drop Columns that won't be useful

df.drop(['SubscriptionId','ProductId','ChannelId','CustomerId', 'CurrencyCode', 'CountryCode'], axis = 1, inplace = True)

In [100]:
# Verify if columns has been successful deleted
df.head()

,AccountId,Amount,BatchId,FraudResult,PricingStrategy,ProductCategory,ProviderId,TransactionId,Value,Year
0,AccountId_3957,1000.0,BatchId_36123,0.0,2,airtime,ProviderId_6,TransactionId_76871,1000,2018
1,AccountId_4841,-20.0,BatchId_15642,0.0,2,financial_services,ProviderId_4,TransactionId_73770,20,2018
2,AccountId_4229,500.0,BatchId_53941,0.0,2,airtime,ProviderId_6,TransactionId_26203,500,2018
3,AccountId_648,20000.0,BatchId_102363,0.0,2,utility_bill,ProviderId_1,TransactionId_380,21800,2018
4,AccountId_4841,-644.0,BatchId_38780,0.0,2,financial_services,ProviderId_4,TransactionId_28195,644,2018


In [101]:
df.tail()

,AccountId,Amount,BatchId,FraudResult,PricingStrategy,ProductCategory,ProviderId,TransactionId,Value,Year
45014,AccountId_4841,-5000.0,BatchId_117348,NaN,2,financial_services,ProviderId_4,TransactionId_116198,5000,2019
45015,AccountId_4803,10000.0,BatchId_53630,NaN,1,airtime,ProviderId_3,TransactionId_24829,10000,2019
45016,AccountId_4817,10000.0,BatchId_112209,NaN,1,airtime,ProviderId_3,TransactionId_102920,10000,2019
45017,AccountId_4841,-5000.0,BatchId_58734,NaN,2,financial_services,ProviderId_4,TransactionId_128439,5000,2019
45018,AccountId_4593,3500.0,BatchId_98308,NaN,4,airtime,ProviderId_5,TransactionId_45927,3500,2019


In [104]:
df['ProductCategory'].describe()

count                 140681
unique                    10
top       financial_services
freq                   67105
Name: ProductCategory, dtype: object

In [105]:
df['ProviderId'].describe()

count           140681
unique               6
top       ProviderId_4
freq             56445
Name: ProviderId, dtype: object

In [110]:
df['PricingStrategy'].describe

<bound method NDFrame.describe of 0        2
1        2
2        2
3        2
4        2
5        2
6        4
7        2
8        2
9        2
10       2
11       2
12       2
13       2
14       2
15       2
16       2
17       2
18       2
19       4
20       2
21       4
22       2
23       2
24       2
25       2
26       2
27       2
28       2
29       2
        ..
44989    2
44990    1
44991    2
44992    4
44993    1
44994    2
44995    1
44996    2
44997    1
44998    1
44999    1
45000    2
45001    1
45002    2
45003    2
45004    2
45005    2
45006    1
45007    2
45008    2
45009    1
45010    2
45011    1
45012    2
45013    1
45014    2
45015    1
45016    1
45017    2
45018    4
Name: PricingStrategy, Length: 140681, dtype: int64>

In [115]:
df['BatchId'].describe()

count            140681
unique           139493
top       BatchId_67019
freq                 28
Name: BatchId, dtype: object

In [0]:
# Encode PricingStrategy column
categorical = ['PricingStrategy']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [121]:
df.head()

,AccountId,Amount,BatchId,FraudResult,ProductCategory,ProviderId,TransactionId,Value,Year,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4
0,AccountId_3957,1000.0,BatchId_36123,0.0,airtime,ProviderId_6,TransactionId_76871,1000,2018,0,0,1,0
1,AccountId_4841,-20.0,BatchId_15642,0.0,financial_services,ProviderId_4,TransactionId_73770,20,2018,0,0,1,0
2,AccountId_4229,500.0,BatchId_53941,0.0,airtime,ProviderId_6,TransactionId_26203,500,2018,0,0,1,0
3,AccountId_648,20000.0,BatchId_102363,0.0,utility_bill,ProviderId_1,TransactionId_380,21800,2018,0,0,1,0
4,AccountId_4841,-644.0,BatchId_38780,0.0,financial_services,ProviderId_4,TransactionId_28195,644,2018,0,0,1,0


In [0]:
# Encode Productcategory column and ProviderId column
category = ['ProductCategory', 'ProviderId']

for var in category:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [0]:
df.drop(['TransactionId'], axis = 1, inplace = True)

In [169]:
df.head()

,Amount,FraudResult,Value,Year,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_retail,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6
0,1000.0,0.0,1000,2018,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-20.0,0.0,20,2018,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,500.0,0.0,500,2018,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,20000.0,0.0,21800,2018,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,-644.0,0.0,644,2018,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
# Create train and test data set from df dataset
train = df[pd.notnull(df['FraudResult'])]
test = df[pd.isnull(df['FraudResult'])].drop(['FraudResult'], axis=1)

In [171]:
train.shape

(95662, 24)

In [172]:
test.shape

(45019, 23)

In [0]:
# Create features(X) and Label (y)
X =train.drop(['FraudResult'], axis = 1)
y = train['FraudResult']

In [0]:
# Spliting the df data to train and test samples
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [175]:
for i in [X_train, X_val, test]:
    print(i.shape)

(76529, 23)
(19133, 23)
(45019, 23)


In [0]:
# Train The Model using Logistic Regression model
log = LogisticRegression(penalty='l2')

In [240]:
# Fitting the log Model
log = log.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
#Testing the model
y_pred = log.predict(test)

Creating A Submission file for Logistic Regression Model

In [0]:
#sub = pd.DataFrame()
sub = test_df[['TransactionId']]

In [250]:
sub['FraudResult'] = y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [194]:
sub.shape


(45019, 2)

In [195]:
sub.tail()

,TransactionId,FraudResult
45014,TransactionId_116198,0.0
45015,TransactionId_24829,0.0
45016,TransactionId_102920,0.0
45017,TransactionId_128439,0.0
45018,TransactionId_45927,0.0


In [0]:
sub.to_csv('submission.csv', index = False)

Trying to Use Random Forest

In [0]:
# Train The Model using Random Forest
rf = RandomForestClassifier(random_state=42)

In [306]:
# Fiting the rf model
rf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
# Predict the y values
y_pred = rf.predict(test)

In [308]:
# test our CV score
cross_val_score(rf, X_train, y_train, cv=5).mean()

0.9996864013327158

Creating A Submission file for Random Forest Model


In [0]:
RFSub = test_df[['TransactionId']]

In [310]:
RFSub['FraaudResult'] = y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
RFSub.to_csv('ForestSub.csv', index = False)

In [312]:
RFSub.head()

,TransactionId,FraaudResult
0,TransactionId_50600,0.0
1,TransactionId_95109,0.0
2,TransactionId_47357,0.0
3,TransactionId_28185,0.0
4,TransactionId_22140,0.0


Random Forest seems to work better than Logistic Regression